In [1]:
data_path = "src/resources/data/credit_card.csv"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("customer_look_alike_modelling").getOrCreate()
df = spark.read.option("header", "true").csv(data_path)
df.limit(5).show()

24/08/26 10:08:18 WARN Utils: Your hostname, dvadmin-ThinkPad-E480 resolves to a loopback address: 127.0.1.1; using 192.168.0.15 instead (on interface wlp5s0)
24/08/26 10:08:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/26 10:08:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
df = df.withColumnRenamed("CLIENTNUM", "customer_id")
df = df.drop("Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2")
df = df.drop("Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1")
df.show()


+-----------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|customer_id|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---

In [4]:
df.columns

['customer_id',
 'Attrition_Flag',
 'Customer_Age',
 'Gender',
 'Dependent_count',
 'Education_Level',
 'Marital_Status',
 'Income_Category',
 'Card_Category',
 'Months_on_book',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio']

In [5]:

# rename to cols_to_convert
# rows_to_convert = "job_titles_cont,city_pl".split(",")
rows_to_convert = df.columns
rows_to_convert.remove("customer_id")
rows_to_convert.remove("Attrition_Flag")
rows_to_convert

['Customer_Age',
 'Gender',
 'Dependent_count',
 'Education_Level',
 'Marital_Status',
 'Income_Category',
 'Card_Category',
 'Months_on_book',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio']

In [6]:
",".join(rows_to_convert)

'Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio'

In [7]:
df.count()

10127

In [8]:
from src.utils.functions import get_row_as_text, hf_embeddings, get_ars_retrieved_df
train_df = get_row_as_text(df, rows_to_convert)

/home/dvadmin/look_alike_modelling_streamlit/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dvadmin/look_alike_modelling_streamlit/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
train_df.select("row_as_text").show(2, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|row_as_text                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
+---------------------------------------------------------------------------------

In [10]:
train_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: string (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: string (nullable = true)
 |-- Total_Relationship_Count: string (nullable = true)
 |-- Months_Inactive_12_mon: string (nullable = true)
 |-- Contacts_Count_12_mon: string (nullable = true)
 |-- Credit_Limit: string (nullable = true)
 |-- Total_Revolving_Bal: string (nullable = true)
 |-- Avg_Open_To_Buy: string (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: string (nullable = true)
 |-- Total_Trans_Amt: string (nullable = true)
 |-- Total_Trans_Ct: string (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: string (nullable = true)
 |-- Avg_Utilization_Ratio: string (nullable 

In [11]:
step = 1000
k = 4000

texts_list = train_df.rdd.collect()
# texts_list = [x[0] for x in texts_label_list]
# texts_list
texts_list[0]

Row(customer_id='768805383', Attrition_Flag='Existing Customer', Customer_Age='45', Gender='M', Dependent_count='3', Education_Level='High School', Marital_Status='Married', Income_Category='$60K - $80K', Card_Category='Blue', Months_on_book='39', Total_Relationship_Count='5', Months_Inactive_12_mon='1', Contacts_Count_12_mon='3', Credit_Limit='12691', Total_Revolving_Bal='777', Avg_Open_To_Buy='11914', Total_Amt_Chng_Q4_Q1='1.335', Total_Trans_Amt='1144', Total_Trans_Ct='42', Total_Ct_Chng_Q4_Q1='1.625', Avg_Utilization_Ratio='0.061', row_as_text='Customer_Age: 45; Gender: M; Dependent_count: 3; Education_Level: High School; Marital_Status: Married; Income_Category: $60K - $80K; Card_Category: Blue; Months_on_book: 39; Total_Relationship_Count: 5; Months_Inactive_12_mon: 1; Contacts_Count_12_mon: 3; Credit_Limit: 12691; Total_Revolving_Bal: 777; Avg_Open_To_Buy: 11914; Total_Amt_Chng_Q4_Q1: 1.335; Total_Trans_Amt: 1144; Total_Trans_Ct: 42; Total_Ct_Chng_Q4_Q1: 1.625; Avg_Utilization_R

In [12]:
import os

# Specify the path of the directory you want to create
db_dir = "src/resources/embeddings/credit"

# Create the directory
os.makedirs(db_dir, exist_ok=True)

In [13]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb
import os

client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

AttributeError: module 'chromadb' has no attribute 'PersistentClient'

In [14]:
from langchain.vectorstores import Chroma
vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings, collection_metadata={"hnsw:space": "cosine"})
for i in range(0, len(texts_list), step):
    texts = [x.row_as_text for x in texts_list[i:i+step]]
    metadata = [{"customer_id": str(x.customer_id)} for x in texts_list[i:i+step]]
    vdb.add_texts(texts, metadata)
    vdb.persist()

In [40]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb

client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

In [16]:
vdb._collection.count()

10127

In [42]:
train_df.count()

10127

In [44]:
df.drop("Attrition_Flag").toPandas().to_csv("src/resources/data/credit_master.csv", header=True, index=False)

In [43]:
df.groupBy("Attrition_Flag").count().show()

+-----------------+-----+
|   Attrition_Flag|count|
+-----------------+-----+
|Existing Customer| 8500|
|Attrited Customer| 1627|
+-----------------+-----+



In [6]:
df = spark.read.option("header", "true").csv("src/resources/data/credit_test.csv").drop("_c0")
df.show()

+-----------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|customer_id|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+------

In [8]:
from pyspark.sql import functions as F
df.filter(F.col("Attrition_Flag") == "Attrited Customer").limit(100).drop("Attrition_Flag").toPandas().to_csv("src/resources/data/credit_test.csv", header=True, index=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Attrition_Flag` cannot be resolved. Did you mean one of the following? [`Education_Level`, `Marital_Status`, `Credit_Limit`, `Customer_Age`, `Gender`].;
'Filter ('Attrition_Flag = Attrited Customer)
+- Project [customer_id#432, Customer_Age#433, Gender#434, Dependent_count#435, Education_Level#436, Marital_Status#437, Income_Category#438, Card_Category#439, Months_on_book#440, Total_Relationship_Count#441, Months_Inactive_12_mon#442, Contacts_Count_12_mon#443, Credit_Limit#444, Total_Revolving_Bal#445, Avg_Open_To_Buy#446, Total_Amt_Chng_Q4_Q1#447, Total_Trans_Amt#448, Total_Trans_Ct#449, Total_Ct_Chng_Q4_Q1#450, Avg_Utilization_Ratio#451]
   +- Relation [_c0#431,customer_id#432,Customer_Age#433,Gender#434,Dependent_count#435,Education_Level#436,Marital_Status#437,Income_Category#438,Card_Category#439,Months_on_book#440,Total_Relationship_Count#441,Months_Inactive_12_mon#442,Contacts_Count_